In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import os

In [3]:
import importlib
import functions
importlib.reload(functions)
functions.set_user('Kaja')
data = functions.call_data_clean(p_threshold=1.3)

MemoryError: Unable to allocate 64.0 KiB for an array with shape (8192,) and data type int64

In [3]:
ATAC = data['ATAC_seq']
RNA = data['RNA_seq']
QC = data['QC_metrics']

In [ ]:
Transcription_exons_path = "D:\\Uni\\4. Semester\\Bioinfo\\datasets\\refFlat.txt"
exons=pd.read_csv(Transcription_exons_path, sep="\t")
exons.columns = ["Genname", "ID", "Chromosom", "Strand", "Transcription Start", "Transcription End", "CDS_Start", "CDS_End", 
    "ExonCount", "ExonStarts", "ExonEnds"]

print(exons.head())

In [ ]:
rna_data=RNA.copy() #damit eingabedaten nicht verändert werden
rna_data=rna_data.T
print(rna_data.head())

In [ ]:
Schritte:

    Nimm ein ATAC-Peak und das zugehörige Gen.

    Hole dir ATAC-Werte für alle Zelltypen (Spalten).

    Hole dir RNA-Werte für dasselbe Gen in denselben Zelltypen.

    Berechne z.B. Pearson-Korrelation.

In [6]:
atac_annotated = pd.read_csv("D:\\Uni\\4. Semester\\Bioinfo\\datasets\\ATAC_dis_annotated.csv")

ParserError: Error tokenizing data. C error: out of memory

In [8]:
# atac_annotated.head()
# atac_annotated['nearest_gene']
print(atac_annotated.index)
print(atac_annotated.head())

RangeIndex(start=0, stop=512590, step=1)
   ImmGenATAC1219.peakID chrom   Summit  mm10.60way.phastCons_scores  \
0  ImmGenATAC1219.peak_1  chr1  3020786                         0.00   
1  ImmGenATAC1219.peak_2  chr1  3087226                         0.00   
2  ImmGenATAC1219.peak_3  chr1  3120109                         0.07   
3  ImmGenATAC1219.peak_4  chr1  3121485                         0.15   
4  ImmGenATAC1219.peak_5  chr1  3372787                         0.03   

   _-log10_bestPvalue  Included.in.systematic.analysis  TSS  \
0                0.56                              NaN  NaN   
1                0.50                              NaN  NaN   
2               10.80                              1.0  NaN   
3                3.02                              1.0  NaN   
4                1.31                              NaN  NaN   

  genes.within.100Kb  LTHSC.34-.BM  LTHSC.34+.BM  ...       var        sd  \
0                NaN      0.271312      0.428712  ...  0.165813  0.407

In [5]:
print(atac_annotated.columns)

NameError: name 'atac_annotated' is not defined

In [4]:
print(atac_annotated['distance_to_TSS'])

NameError: name 'atac_annotated' is not defined

In [ ]:
atac_annotated = atac_annotated.copy()
atac_annotated=atac_annotated.T
#print(atac_annotated.head())
atac_annotated.set_index(atac_annotated.columns[0], inplace=True)
print(atac_annotated.index)
print(atac_annotated.head())
rna_data = RNA.copy()
rna_data=rna_data.T
print(rna_data.index)

nearest_gene_name=atac_annotated['nearest_gene']

gemeinsame_zelltypen = list((atac_annotated.index).intersection(rna_data.index))

print(f"Anzahl gemeinsamer Zelltypen: {len(gemeinsame_zelltypen)}")
print("Gemeinsame Zelltypen (Beispiel):", list(gemeinsame_zelltypen)[:10])

# Zelltypen, die nur in einem der DataFrames vorkommen
print("\nZelltypen nur in ATAC:", list(set(atac_annotated.index) - set(rna_data.index))[:10])
print("Zelltypen nur in QC:", list(set(rna_data.index) - set(atac_annotated.index))[:10])


#r, p_values = pearsonr(atac_annotated, rna_data)

In [13]:
import pandas as pd
from scipy.stats import pearsonr
import numpy as np

def correlate_atac_with_rna(atac_df, rna_df, gene_column="nearest_gene"):
    """
    Für jeden Peak mit bekanntem Gen wird die Pearson-Korrelation zwischen ATAC und RNA berechnet.
    
    Parameters:
    atac_df: DataFrame mit Peaks als Zeilen, Zelltypen als Spalten, und einer Spalte mit Genname (z.B. 'nearest_gene')
    rna_df: DataFrame mit Genen als Zeilen (Index), Zelltypen als Spalten (müssen mit ATAC übereinstimmen)
    gene_column: Name der Spalte in atac_df, die das zugehörige Gen enthält
    
    Returns:
    korrelationen_df: DataFrame mit peak_id, zugehörigem Gen, r (Korrelation), p-Wert
    """
    
    # Zelltyp-Spalten extrahieren (nur numerische, keine Strings wie nearest_gene)
    celltypes = atac_df.select_dtypes(include=[np.number]).columns
    print(f"Verwendete Zelltypen: {list(celltypes)}")
    
    results = []
    
    for idx, row in atac_df.iterrows():
        gene = row[gene_column]
        if pd.isna(gene) or gene not in rna_df.index:
            continue
        
        atac_values = row[celltypes].values
        rna_values = rna_df.loc[gene, celltypes].values
        
        # Prüfe auf NaNs oder konstante Werte
        if np.isnan(atac_values).any() or np.isnan(rna_values).any():
            continue
        if np.std(atac_values) == 0 or np.std(rna_values) == 0:
            continue
        
        r, p = pearsonr(atac_values, rna_values)
        
        results.append({
            "peak_id": idx,
            "gene": gene,
            "r": r,
            "p_value": p
        })
    
    korrelationen_df = pd.DataFrame(results)
    return korrelationen_df


In [6]:
korrelation rna und atac
distance to Tss
je kleiner distanz desto höher korrelation?
korreliere abstand mit expression und rna mit peaks
größe der rna atac korrelaiton gegen distance
haben alle promotoren einen hohen korrelationswert bei atac Rna
je näher dran desto stärker wirkt der promotor?
korrelieren manche promotoren mit mehreren genen, auch die nicht in der nähe liegen
filtern nach großer korrelation und großer abstand
welche werte woher und welches format

SyntaxError: invalid syntax (3678462444.py, line 1)

In [15]:
correlate_atac_with_rna(atac_df=df, rna_df=RNA, gene_column="nearest_gene")

Verwendete Zelltypen: ['Summit', 'mm10.60way.phastCons_scores', '_-log10_bestPvalue', 'Included.in.systematic.analysis', 'LTHSC.34-.BM', 'LTHSC.34+.BM', 'STHSC.150-.BM', 'MPP4.135+.BM', 'proB.CLP.BM', 'proB.FrA.BM', 'proB.FrBC.BM', 'preB.FrD.BM', 'B.FrE.BM', 'B1b.PC', 'B.T1.Sp', 'B.T2.Sp', 'B.T3.Sp', 'B.Sp', 'B.Fem.Sp', 'B.MZ.Sp', 'B.Fo.Sp', 'B.mem.Sp', 'B.GC.CB.Sp', 'B.GC.CC.Sp', 'B.PB.Sp', 'B.PC.Sp', 'B.PC.BM', 'preT.DN1.Th', 'preT.DN2a.Th', 'preT.DN2b.Th', 'preT.DN3.Th', 'T.DN4.Th', 'T.ISP.Th', 'T.DP.Th', 'T.4.Th', 'T.8.Th', 'T.4.Nve.Sp', 'T.4.Nve.Fem.Sp', 'T.8.Nve.Sp', 'T.4.Sp.aCD3+CD40.18hr', 'Treg.4.FP3+.Nrplo.Co', 'Treg.4.25hi.Sp', 'T8.TN.P14.Sp', 'T8.IEL.LCMV.d7.SI', 'T8.TE.LCMV.d7.Sp', 'T8.MP.LCMV.d7.Sp', 'T8.Tcm.LCMV.d180.Sp', 'T8.Tem.LCMV.d180.Sp', 'NKT.Sp', 'NKT.Sp.LPS.3hr', 'NKT.Sp.LPS.18hr', 'NKT.Sp.LPS.3d', 'Tgd.g2+d17.24a+.Th', 'Tgd.g2+d17.LN', 'Tgd.g2+d1.24a+.Th', 'Tgd.g2+d1.LN', 'Tgd.g1.1+d1.24a+.Th', 'Tgd.g1.1+d1.LN', 'Tgd.Sp', 'NK.27+11b-.BM', 'NK.27+11b+.BM', 'NK.2

KeyError: "['Summit', 'mm10.60way.phastCons_scores', '_-log10_bestPvalue', 'Included.in.systematic.analysis', 'preB.FrD.BM', 'T8.IEL.LCMV.d7.SI', 'MF.LP.SI', 'DC.103+11b+.SI', 'DC.103+11b-.SI', 'distance_to_TSS', 'mean', 'median', 'var', 'sd', 'variability', 'min', 'max', 'range', 'skewness'] not in index"

In [9]:
from scipy.stats import pearsonr
atac_data=ATAC.copy()

    
atac_data=atac_data.select_dtypes(include='number')
atac_data=atac_data.T
#print(atac_data.head())
atac_data.drop(['Summit', 'mm10.60way.phastCons_scores', '_-log10_bestPvalue', 'Included.in.systematic.analysis'], axis=0)
#print(atac_data.index)
#print(atac_data.columns)
rna_data=RNA.copy() #damit eingabedaten nicht verändert werden
rna_data=rna_data.T
print(rna_data.index)
#Index festlegen

#rna_data.set_index(rna_data.columns[0], inplace=True)
print(rna_data.index)
# print(qc_data.columns)

print(atac_data.shape)
print(rna_data.shape)
print(rna_data.head())

#atac_data.set_index('ImmGenATAC1219.peakID', inplace=True)
#atac_data.set_index(atac_data.columns[0], inplace=True)
# print(qc_data.index)

# print(atac_data.index)

# Einheitliche Formatierung der Zelltypen
# atac_data.columns = atac_data.columns.astype(str).str.lower().str.replace(" ", "")
# qc_data.columns = qc_data.columns.astype(str).str.lower().str.replace(" ", "")

#qc_data.index = qc_data.index.astype(str).str.lower().str.replace(" ", "")

# Gemeinsame Zelltypen identifizieren
#gemeinsame_zelltypen = set(atac_data.index).intersection(qc_data.index)
gemeinsame_zelltypen = list((atac_data.index).intersection(rna_data.index))

print(f"Anzahl gemeinsamer Zelltypen: {len(gemeinsame_zelltypen)}")
print("Gemeinsame Zelltypen (Beispiel):", list(gemeinsame_zelltypen)[:10])

# Zelltypen, die nur in einem der DataFrames vorkommen
print("\nZelltypen nur in ATAC:", list(set(atac_data.index) - set(rna_data.index))[:10])
print("Zelltypen nur in QC:", list(set(rna_data.index) - set(atac_data.index))[:10])

# atac_stat=atac_data.mean(axis=1)#.rename('ATAC_stat')


# #common_cells = atac_stat.index.intersection(qc_data.index)
# atac_vals = atac_stat.loc[gemeinsame_zelltypen]
# #print(atac_vals.head(10))
# print(atac_vals.shape)

# qc_metric_col = 'InputCellNumber'
# qc_data_numeric = qc_data.select_dtypes(include='number')
# qc_data_grouped = qc_data_numeric.groupby(qc_data.index).mean()
# qc_vals = qc_data_grouped.loc[gemeinsame_zelltypen, qc_metric_col]
# print(qc_vals.head())
# print(qc_vals.shape)
# #print(qc_data.index.value_counts())
# #print(qc_vals.head(10))
# # print("ATAC-Werte Länge:", len(atac_vals))
# # print("QC-Werte Länge:", len(qc_vals))
# # print("Fehlende Werte in QC:", qc_vals.isna().sum())
# # print("Fehlende Werte in ATAC:", atac_vals.isna().sum())

# r, p_value=pearsonr(atac_vals, qc_vals)
# print(f"Pearson r = {r:.3f}, p = {p_value:.3g}")

Index(['LTHSC.34-.BM', 'LTHSC.34+.BM', 'STHSC.150-.BM', 'MPP4.135+.BM',
       'proB.CLP.BM', 'proB.FrA.BM', 'proB.FrBC.BM', 'B.FrE.BM', 'B1b.PC',
       'B.T1.Sp', 'B.T2.Sp', 'B.T3.Sp', 'B.Sp', 'B.Fem.Sp', 'B.MZ.Sp',
       'B.Fo.Sp', 'B.mem.Sp', 'B.GC.CB.Sp', 'B.GC.CC.Sp', 'B.PB.Sp', 'B.PC.Sp',
       'B.PC.BM', 'preT.DN1.Th', 'preT.DN2a.Th', 'preT.DN2b.Th', 'preT.DN3.Th',
       'T.DN4.Th', 'T.ISP.Th', 'T.DP.Th', 'T.4.Th', 'T.8.Th', 'T.4.Nve.Sp',
       'T.4.Nve.Fem.Sp', 'T.8.Nve.Sp', 'T.4.Sp.aCD3+CD40.18hr',
       'Treg.4.FP3+.Nrplo.Co', 'Treg.4.25hi.Sp', 'T8.TN.P14.Sp',
       'T8.IEL.LCMV.d7.Gut', 'T8.TE.LCMV.d7.Sp', 'T8.MP.LCMV.d7.Sp',
       'T8.Tcm.LCMV.d180.Sp', 'T8.Tem.LCMV.d180.Sp', 'NKT.Sp',
       'NKT.Sp.LPS.3hr', 'NKT.Sp.LPS.18hr', 'NKT.Sp.LPS.3d',
       'Tgd.g2+d17.24a+.Th', 'Tgd.g2+d17.LN', 'Tgd.g2+d1.24a+.Th',
       'Tgd.g2+d1.LN', 'Tgd.g1.1+d1.24a+.Th', 'Tgd.g1.1+d1.LN', 'Tgd.Sp',
       'NK.27+11b-.BM', 'NK.27+11b+.BM', 'NK.27-11b+.BM', 'NK.27+11b-.Sp',
       '

In [ ]:
cor_df = correlate_promoter_atac_rna(atac_annotated[atac_annotated.index.isin(ATAC_top)], RNA, ATAC_scores)
print(cor_df.columns)
print(cor_df['peak_id'])

In [ ]:
heatmap_df = cor_df.pivot(index='gene', columns='peak_id', values='r')

plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_df, cmap='coolwarm', center=0, vmin=-1, vmax=1)
plt.title('Promoter-ATAC-RNA Pearson-Korrelationen')
plt.xlabel('Peak-ID')
plt.ylabel('Gen')
plt.tight_layout()
plt.show()

In [4]:
def correlate_promoter_atac_rna(ATAC, RNA, ATAC_scores):
    # Nur gemeinsame Cell-Typ-Namen verwenden!
    gemeinsame_celltypen = [col for col in ATAC_scores.columns if col in RNA.columns]
    print(f"Gemeinsame Celltypen ({len(gemeinsame_celltypen)}):", gemeinsame_celltypen)
    
    promoter_peaks = ATAC[ATAC['distance_to_TSS'] <= 2000].copy()
    promoter_peaks['main_gene'] = promoter_peaks['nearest_gene']
    results = []
    for idx, row in promoter_peaks.iterrows():
        gene = row['main_gene']
        if gene in RNA.index:
            atac_vec = row[gemeinsame_celltypen].values.astype(float)
            rna_vec = RNA.loc[gene, gemeinsame_celltypen].values.astype(float)
            if not (np.isnan(atac_vec).any() or np.isnan(rna_vec).any()):
                r, p = scipy.stats.pearsonr(atac_vec, rna_vec)
                results.append({'peak_id': idx, 'gene': gene, 'r': r, 'p': p})
    print(f"Anzahl berechnete Korrelationen: {len(results)}")
    return pd.DataFrame(results)